In [1]:
import h5py as h5
#from h5glance import H5Glance
import numpy as np
import gvar as gv
import pandas as pd
import os 
import sys
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib notebook
#import h5ls
import lsqfit
import matplotlib
from matplotlib.backends.backend_pdf import PdfPages
import yaml
from pathlib import Path
import importlib

In [2]:
#import local files
#sys.path.append("../")
# import data_loader as dl
# import fitter as fit
# import fit_manager as fm
import gmo_fitter as gmo 
import gmo_fit_analysis as fa
import mass_relations as ma 
import i_o 
import load_data_priors as ld

matplotlib.rcParams['figure.figsize'] = [10, 8]

In [3]:
import importlib
fit_params = 'input_files/a09m135.py'
sys.path.append(os.path.dirname(os.path.abspath(fit_params)))
fp = importlib.import_module(
    fit_params.split('/')[-1].split('.py')[0])
p_dict = fp.p_dict

In [13]:
file = '/Users/grantdb/lqcd/data/c51_2pt_octet_decuplet.h5'
with h5.File(file,"r") as f:
    ensembles = {}
    ensembles = list(f.keys())
o_data = ld.get_corr(file,p_dict['abbr'])
# print(o_data)
tmp = {}
for part in ['sigma_p','xi_z','proton','lambda_z']:

    for i,s in enumerate(p_dict['snks']):
        tmp[(part,s)] = o_data[part][:,:,i,0]
    # lam_corr[s] = o_data['lambda_z'][:,:,i,0]
    # xi_corr[s] = o_data['xi_z'][:,:,i,0]
    # sigma_corr[s] = o_data['sigma_p'][:,:,i,0]
    # piplus_corr[s] = o_data['piplus'][:,:,i,0]
    # kplus_corr[s] = o_data['kplus'][:,:,i,0]

# nucleon_corr_gv = gv.dataset.avg_data(nucleon_corr)
# lam_corr_gv = gv.dataset.avg_data(lam_corr)
# xi_corr_gv = gv.dataset.avg_data(xi_corr)
# sigma_corr_gv = gv.dataset.avg_data(sigma_corr)
# piplus_corr_gv = gv.dataset.avg_data(piplus_corr)
# kplus_corr_gv = gv.dataset.avg_data(kplus_corr)
temp = gv.dataset.avg_data(tmp)
temp

BufferDict({
    ('sigma_p', 'SS'):
    array([6.781(54)e-09, 3.4616(57)e-07, 1.1259(24)e-07, 4.926(11)e-08,
           2.4240(57)e-08, 1.2692(30)e-08, 6.896(17)e-09, 3.839(10)e-09,
           2.1730(63)e-09, 1.2443(39)e-09, 7.188(25)e-10, 4.177(16)e-10,
           2.436(11)e-10, 1.4241(72)e-10, 8.341(49)e-11, 4.895(33)e-11,
           2.879(23)e-11, 1.697(17)e-11, 1.001(12)e-11, 5.919(86)e-12,
           3.517(63)e-12, 2.100(47)e-12, 1.252(35)e-12, 7.45(27)e-13,
           4.44(20)e-13, 2.63(15)e-13, 1.55(12)e-13, 9.17(89)e-14,
           5.45(68)e-14, 3.23(53)e-14, 1.87(42)e-14, 1.03(33)e-14,
           5.7(2.6)e-15, 3.1(2.0)e-15, 1.3(1.6)e-15, 2(12)e-16, -4.0(9.9)e-16,
           -6.0(8.1)e-16, -5.6(6.6)e-16, -4.8(5.4)e-16, -3.4(4.4)e-16,
           -1.6(3.6)e-16, -8(29)e-17, -8(24)e-17, -5(19)e-17, -3(16)e-17,
           -9(136)e-18, -3(117)e-18, 3(1019)e-19, 4.6(8.9)e-17, 7.5(7.9)e-17,
           3.5(7.4)e-17, -1(73)e-18, -1.7(7.7)e-17, -4.6(8.7)e-17, -7(10)e-17,
           -4(12)

**

In [5]:
gmo_fits = {}  # C_2pt(t) for each baryon at each time slice t
corr_fits = {} # resulting fit params, need E0 for gmo study 
mass = {}
gmo_states = ['sigma','xi','proton','lam']

for ens in ensembles:
    mass[p_dict['abbr']] = {}

out_path = 'fit_results/{0}/'.format(p_dict['abbr'])

for corr in gmo_states:
    gmo_fits = gv.load(out_path+'fit_data')
    corr_fits = gv.load(out_path+'fit_params')
    mass[p_dict['abbr']][corr] = corr_fits['p'][corr+'_E0']

    # corr_fits[corr]['p']['E0']
gmo_fits
mass

{'a09m135': {'sigma': 0.5337(10),
  'xi': 0.57531(65),
  'proton': 0.4265(15),
  'lam': 0.4931(12)}}

In [6]:
abbr =p_dict['abbr']
# for key in gmo_fits.keys():
nt = 25
t = np.array(np.arange(nt))
nt_ = len(t)
# gmo = {}
gmo_raw_ps =  ma.GMO(sigma=sigma_corr['PS'],lam=lam_corr['PS'],
nucleon=nucleon_corr['PS'],xi=xi_corr['PS'],t=nt_)
gmo_raw_ss =  ma.GMO(sigma=sigma_corr['SS'],lam=lam_corr['SS'],
nucleon=nucleon_corr['SS'],xi=xi_corr['SS'],t=nt_)

gmo_ps = ma.GMO(sigma=gmo_fits['sigma_PS'],lam=gmo_fits['lam_PS'],
nucleon=gmo_fits['nucleon_PS'],xi=gmo_fits['xi_PS'],t=nt_)
gmo_ss = ma.GMO(sigma=gmo_fits['sigma_SS'],lam=gmo_fits['lam_SS'],
nucleon=gmo_fits['nucleon_SS'],xi=gmo_fits['xi_SS'],t=nt_)

gmo_violation = ma.GMO(sigma=mass[abbr]['sigma'],lam=mass[abbr]['lam'],
nucleon=mass[abbr]['proton'],xi=mass[abbr]['xi'],t=nt_)
gmo_out = {}
gmo_raw_out = {}
gmo_raw_out['PS'] = gmo_raw_ps._G_gmo(log=False)
gmo_raw_out['SS'] = gmo_raw_ss._G_gmo(log=False)
# gmo_out['PS'] = gmo_ps._G_gmo(log=False)
# gmo_out['SS']= gmo_ss._G_gmo(log=False)

gmo_raw_out
# print(gmo_violation._gmo_violation())



/Users/grantdb/lqcd/gmo_relation/mass_relations.py:138: RuntimeWarning: invalid value encountered in power
  (self.lam[T] * (self.sigma[T])**1/3) * (self.nucleon[T]**(-2/3))*  (self.xi[T])**(-2/3)


ValueError: Inconsistent array shapes or data types in dataset.

In [ ]:
eff_mass = ma.gmo_eff_mass(gmo_out=gmo_out,dt=1)
eff_mass

{'PS': array([1.5147(31), 1.4880(25), 1.4691(24), 1.4562(22), 1.4476(21),
        1.4420(21), 1.4383(22), 0.06908(68)], dtype=object),
 'SS': array([1.4823(19), 1.4652(18), 1.4536(18), 1.4459(18), 1.4409(20),
        1.4376(21), 1.4355(22), 0.07368(61)], dtype=object)}

In [ ]:
ma.plot_log_gmo(gmo_out ,0, 7,fig_name='gmo_test')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
ma.plot_effective_mass(eff_mass,t_plot_min=1,t_plot_max=8,fig_name='eff_mass')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>